# Event Sampling for Gammapy


## Introduction
[Gammapy](https://gammapy.org/) is a Python package for gamma-ray astronomy. Based on the scientific Python stack ([Numpy](http://www.numpy.org/), [Scipy](https://www.scipy.org/) and [Astropy](http://www.astropy.org/)) it implements high level tools for gamma-ray data analysis, such as combined fitting of spectral and morphological models,
joint-likelihood analyses, image bases analysis methods, timing analysis etc. It started ~5 years ago as a loose collection of analysis scripts needed for the HESS Galactic Plane Survey, but as now evolved into a serious analysis
package, already used by >20 people. The main goal is to establish Gammapy as the future CTA science tools. 


## Gammapy Development
The Gammapy project is not only open-source, but developed openly in a [github repository](https://github.com/gammapy/gammapy). The project documentation can be found on https://docs.gammapy.org/0.10/. There are 3 lead developers ([Christoph Deil](https://github.com/cdeil), [Regis Terrier](https://github.com/registerrier) and [me](https://github.com/adonath)) and ~10 people that have contributed regularly in the past and some are still contributing. Contributions are made by "pull requests", that are typically review by the lead developers.

The timeline for Gammapy development is set by the CTA science tools selection process. We have defined as one of our major milestones a Gammapy 1.0 release for late 2019. To coordinate the required development for this we have written a [Gammapy Roadmap 2019](https://github.com/gammapy/gammapy/blob/b027fe3fef1ad325bb89368b57218bd8a5a4b644/docs/development/pigs/pig-005.rst) document. 

Large code contributions to Gammapy are outlined and planned in document called "proposal for improving Gammapy" (PIG).
The current PIGs are listed [here](https://docs.gammapy.org/dev/development/pigs/index.html). A PIG outlines the reasoning for adding new functionality, makes a proposal what and how to implement and ideally breaks the work into a list of smaller code contributions ("pull requests") (see e.g. recent PIGs on [Datasest](https://github.com/gammapy/gammapy/pull/1986) and [Models](https://github.com/gammapy/gammapy/pull/1971)). It also defines the timeline of the 
pull requests in terms of inermediate milestones such as `v0.11`, `v0.12`, etc.

The ideal outcome of this meeting would be a **draft for a PIG, that addresses event sampling in Gammapy**. The PIG would be improved in the coming weeks and once it's considered "final", the actual implementation would start. 

## Requirements
For the purpose simulating gamma-ray data, event sampling is listed as one of the requirements for the CTA science tools. For the first CTA data challenge data has been simulated by [gammalib](http://gammalib.sourceforge.net/) / [ctools](http://cta.irap.omp.eu/ctools/). One goal for Gammapy could be to contribute simulated data to the next data challenge.
This means simulating data at a scale of thousands of observations with hundred of thousands of events each.

The main parameters to sample for the events are:

- Reconstructed arrival direction in sky coordinates (`ra`, `dec`)
- Reconstructed energy
- Arrival time
- Component ID (unique identifier for the model component)
- (Event class) 
- (Event type)
- (True direction)
- (True energy)
- (Arrival direction in camera coordinates, (`DETX`, `DETY`))

The complexity of the source models is unclear. It might include energy-dependent morphologies, time dependent spectra, pulsar phases etc. If possible we should try to avoid restrictions on this by design.

The implementation best relies on Numpy, Scipy, Astropy and Gammapy, but could be extended to using e.g. cython if performance becomes an issue (e.g. when loops over events are required). 


## Implementation Proposal
Gammapy only implements "binned analysis", i.e. events are typically binned into 1D, 2D, or ND data structures https://docs.gammapy.org/dev/notebooks/intro_maps.html) and not processed as individual events in event lists.
So an obvious approach to implementing the event sample is based on "binned" predicted numbers of counts. 

Some more ideas are outline here:

* https://github.com/gammapy/gammapy/issues/761
* https://github.com/gammapy/gammapy/issues/74


Further references to look at:
* http://cta.irap.omp.eu/ctools/users/reference_manual/ctobssim.html
* Any other code?

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from astropy.coordinates import SkyCoord
from astropy.table import Table

from gammapy.maps import Map, MapAxis, MapCoord
from gammapy.data import EventList
from gammapy.cube import fill_map_counts
from gammapy.utils.random import get_random_state

### Example Event List from CTA 1DC

For the first data challenge events were sampled by 

In [2]:
events = EventList.read("$GAMMAPY_DATA/cta-1dc/data/baseline/gps/gps_baseline_110380.fits")
events.table

EVENT_ID,TIME,RA,DEC,ENERGY,DETX,DETY,MC_ID
,s,deg,deg,TeV,deg,deg,
uint32,float64,float32,float32,float32,float32,float32,int32
1,664502403.0454683,-92.63541,-30.514854,0.03902182,-0.9077294,-0.2727693,2
2,664502405.2579999,-92.64103,-28.262728,0.030796371,1.3443842,-0.2838398,2
3,664502408.8205513,-93.20372,-28.599625,0.04009629,1.0049409,-0.7769775,2
4,664502409.0143764,-94.03383,-29.269627,0.039580025,0.32684833,-1.496021,2
5,664502414.8090746,-93.330505,-30.319725,0.03035851,-0.716062,-0.8733348,2
6,664502415.5855484,-93.23232,-28.587324,0.034782063,1.0170497,-0.8021856,2
7,664502416.0332305,-92.62048,-29.781712,0.04999659,-0.17455244,-0.26183704,2
8,664502417.712146,-93.75603,-30.201115,0.041633684,-0.6013596,-1.242136,2


### Toy Event Sampler

This is a super simple event sample that I've written, based on inverse CDF sampling (code adapted from https://stackoverflow.com/questions/21100716/fast-arbitrary-distribution-random-sampling/21101584#21101584), that illustrated the basic idea and might serve as a starting point.

In [3]:
class MapEventSampler(object):
    """
    Draw samples from an npred map.
    
    Parameters
    ----------
    npred : `Map`
        Predicted number of counts map, specifying the propability
        density function.
    random_state : int
        Random state
    interpolation : bool
        Use interpolation
    """
    def __init__(self, npred, random_state=0, interpolation=True):
        self.npred = npred
        self.interpolation = interpolation
        self.random_state = get_random_state(random_state)

        pdf = npred.data.ravel()    
        self.sortindex = np.argsort(pdf, axis=None)

        self.pdf = pdf[self.sortindex]
        self.cdf = np.cumsum(self.pdf)    

        
    @property
    def npred_total(self):
        """Total number of predicted events"""
        return self.npred.data.sum()
         
    def sample_total_events(self):
        """Sample total number of events."""
        return self.random_state.poisson(self.npred_total)
        
    def _result_table(self, coords):
        lon, lat, energy = coords
        skycoord = SkyCoord(lon, lat, frame="galactic", unit="deg")
        
        table = Table()
        table['EVENT_ID'] = np.arange(len(lon))
        table['RA'] = skycoord.icrs.ra
        table['DEC'] = skycoord.icrs.dec
        table["ENERGY"] = energy * self.npred.geom.get_axis_by_name("energy").unit.to("TeV")
        return table
    
    def run(self):
        """Sample event list.
        
        Returns
        -------
        events : `EventList`
            Sampled events.
        """
        N = self.sample_total_events()
        
        #pick numbers which are uniformly random over the cumulative distribution function
        choice = self.random_state.uniform(high=self.npred_total, size=N)
        
        #find the indices corresponding to this point on the CDF
        index = np.searchsorted(self.cdf, choice)
        index = self.sortindex[index]
        
        #map back to multi-dimensional indexing
        index = np.unravel_index(index, self.npred.data.shape)
        index = np.vstack(index)
        
        #is this a discrete or piecewise continuous distribution?
        if self.interpolation:
            index = index + self.random_state.uniform(low=-0.5, high=0.5, size=index.shape)
        
        coords = self.npred.geom.pix_to_coord(index[::-1])
        table = self._result_table(coords)
        return EventList(table)

In [4]:
sampler = MapEventSampler(background, random_state=0, interpolation=True)

NameError: name 'background' is not defined

In [ ]:
events = sampler.run()

In [ ]:
counts = Map.from_geom(background.geom)
fill_map_counts(counts, events)

In [ ]:
counts.sum_over_axes().smooth("0.1 deg").plot();